In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize


# -------------------------------------------------------
# Paramètres physiques
# -------------------------------------------------------
f = 40000          # fréquence (Hz)
c0 = 343           # vitesse du son dans l'air (m/s)
ω = 2 * np.pi * f
k = ω / c0         # nombre d'onde

# ---- Constantes physiques ----
k = 2 * np.pi * 40000 / 343
R_p = 0.0005
c_p = 2400
rho_p = 28.59
c_0 = 343
rho_0 = 1.204
R_t = 0.005  # rayon transducteur

#---------------------------------------------------------
eps = 1e-4  # pas de dérivation

omega = 2 * np.pi * 40000
k1 = 4 * np.pi/3 * R_p**3 / 4 * (1/(c_0**2*rho_0) - 1/(c_p**2*rho_p))
k2 = 3*4*np.pi/3*R_p**3/4 * ((rho_0 - rho_p)/(omega**2 * rho_0 * (rho_0 + 2*rho_p)))

# -------------------------------------------------------
# Espace de simulation
# -------------------------------------------------------
N =8 # nombre de transducteur par axe (8x8)
a=0.01


# bornes symétriques : +/-5 cm latéralement, 8 cm en hauteur
Lx = 0.05
Ly = 0.05
DzT=0.08

x = np.linspace(-Lx, Lx, N)
y = np.linspace(-Ly, Ly, N)

X_T_b, Y_T_b = np.meshgrid(x, y)
Z_T_b=np.array([[0]*N]*N)

X_T_h, Y_T_h = np.meshgrid(x, y)
Z_T_h=np.array([[DzT]*N]*N)


##on passe tout à une dimension, dans le ref de la plaque :
X_T_b=X_T_b.flatten()
Y_T_b=Y_T_b.flatten()
Z_T_b=Z_T_b.flatten()

X_T_h=X_T_h.flatten()
Y_T_h=Y_T_h.flatten()
Z_T_h=Z_T_h.flatten()

X_T=np.concatenate((X_T_b,X_T_h))
Y_T=np.concatenate((Y_T_b,Y_T_h))
Z_T=np.concatenate((Z_T_b,Z_T_h))

In [2]:
 # === Fonctions utiles ===

def D_F_vectorized(xj, yj, zj, x, y, z):
    theta = np.arctan(np.sqrt((x - xj)**2 + (y - yj)**2) / np.where(np.abs(z - zj) < 1e-12, 1e-12, np.abs(z - zj)))
    df = np.sin(k * R_p * np.sin(theta)) / np.where(np.sin(k * R_p * np.sin(theta)) == 0, 1e-12, k * R_p * np.sin(theta))
    df = np.where(np.isnan(df), 1, df)
    return df

DJ={}
DF={}
def precompute_field_params(x, y, z):
    # Plaque du bas
    if (x,y,z) in DJ:
        return (DJ[(x,y,z)],DF[(x,y,z)])

    else :
      dx = X_T - x
      dy = Y_T - y
      dz = Z_T - z
      dj = np.sqrt(dx**2 + dy**2 + dz**2)
      df = D_F_vectorized(X_T, Y_T, Z_T, x, y, z)
      DJ[(x,y,z)]=dj
      DF[(x,y,z)]=df

      return (dj, df)

# === Calcul du champ total avec dj/df pré-calculés ===

def p_tot(phi_vals, x, y, z):
    dj, df = precompute_field_params(x, y, z)
    ptot = np.sum(df / dj * np.exp(1j * (k*dj + phi_vals)))
    return ptot

# === Gradient numérique ===

def grad_p(phi_vals, x, y, z):
    dp_dx = (p_tot(phi_vals, x+eps, y, z) - p_tot(phi_vals, x,y,z)) / eps
    dp_dy = (p_tot(phi_vals, x, y+eps, z) - p_tot(phi_vals, x,y,z)) / eps
    dp_dz = (p_tot(phi_vals, x, y, z+eps) - p_tot(phi_vals, x,y,z)) / eps
    return np.array([dp_dx, dp_dy, dp_dz])

# === Potentiel de Gorkov ===

def gorkov_potential(phi_vals, x, y, z):
    p_complex = p_tot(phi_vals, x,y,z)
    gradp = grad_p(phi_vals, x, y, z)

    return k1*abs(p_complex)**2 - k2*np.sum(np.abs(gradp)**2)

# === Laplacien du potentiel ===

def lap_U(phi_vals, x, y, z):
    U0 = gorkov_potential(phi_vals, x, y, z)
    Ux = gorkov_potential(phi_vals, x+eps, y, z)
    Uy = gorkov_potential(phi_vals, x, y+eps, z)
    Uz = gorkov_potential(phi_vals, x, y, z+eps)
    return (Ux - 2*U0 + gorkov_potential(phi_vals, x-eps, y, z))/eps**2 + \
           (Uy - 2*U0 + gorkov_potential(phi_vals, x, y-eps, z))/eps**2 + \
           (Uz - 2*U0 + gorkov_potential(phi_vals, x, y, z-eps))/eps**2

# === Fonction objectif pour l’optimisation ===

def F_opt(phi_vals, xf1, yf1, zf1,xf2, yf2, zf2):
    ptot1 = p_tot(phi_vals, xf1, yf1, zf1)
    lapU1 = lap_U(phi_vals, xf1, yf1, zf1)
    ptot2=p_tot(phi_vals, xf2, yf2, zf2)
    lapU2=lap_U(phi_vals, xf2, yf2, zf2)
    ptot=ptot1+ptot2
    lapU=lapU1+lapU2
    return abs(ptot) - lapU

# === optimisation ===

def optimizer(xf1, yf1, zf1,xf2, yf2, zf2):

    def objective(phi_vals):
        phi_mod = np.mod(phi_vals, 2*np.pi)
        return F_opt(phi_mod, xf1, yf1, zf1, xf2, yf2, zf2)

    phi0 = np.zeros(2*(N**2))
    result = minimize(objective, phi0, method='L-BFGS-B')
    phases_mod = np.mod(result.x, 2*np.pi)

    print(", ".join(f"{a:.4f}" for a in phases_mod))
    return phases_mod

In [3]:
# === Point de focalisation ===
X_foc1, Y_foc1, Z_foc1 = 0.01, 0, 0.03
X_foc2, Y_foc2, Z_foc2 = -0.01, 0, 0.03

# === Lancement ===
optimizer(X_foc1, Y_foc1, Z_foc1, X_foc2, Y_foc2, Z_foc2)

6.2759, 6.2074, 0.1549, 0.1131, 0.1131, 0.1549, 6.2074, 6.2759, 0.0464, 6.2157, 6.2356, 0.0016, 0.0016, 6.2356, 6.2157, 0.0464, 6.1106, 0.0030, 6.2050, 6.2325, 6.2325, 6.2050, 0.0030, 6.1106, 0.2066, 6.2468, 0.1613, 6.2801, 6.2801, 0.1613, 6.2468, 0.2066, 0.2066, 6.2468, 0.1613, 6.2801, 6.2801, 0.1613, 6.2468, 0.2066, 6.1106, 0.0030, 6.2050, 6.2325, 6.2325, 6.2050, 0.0030, 6.1106, 0.0464, 6.2157, 6.2356, 0.0016, 0.0016, 6.2356, 6.2157, 0.0464, 6.2759, 6.2074, 0.1549, 0.1131, 0.1131, 0.1549, 6.2074, 6.2759, 0.0065, 0.1421, 6.2368, 6.1482, 6.1482, 6.2368, 0.1421, 0.0065, 0.0095, 0.1502, 6.2260, 6.1533, 6.1533, 6.2260, 0.1502, 0.0095, 0.0263, 6.2615, 6.0938, 6.1915, 6.1915, 6.0938, 6.2615, 0.0263, 6.2077, 0.0794, 0.1401, 0.0447, 0.0447, 0.1401, 0.0794, 6.2077, 6.2077, 0.0794, 0.1401, 0.0447, 0.0447, 0.1401, 0.0794, 6.2077, 0.0263, 6.2615, 6.0938, 6.1915, 6.1915, 6.0938, 6.2615, 0.0263, 0.0095, 0.1502, 6.2260, 6.1533, 6.1533, 6.2260, 0.1502, 0.0095, 0.0065, 0.1421, 6.2368, 6.1482, 6.1482, 

array([6.27593158e+00, 6.20738473e+00, 1.54858542e-01, 1.13146827e-01,
       1.13146826e-01, 1.54858541e-01, 6.20738473e+00, 6.27593158e+00,
       4.63613553e-02, 6.21567427e+00, 6.23559348e+00, 1.64551988e-03,
       1.64551456e-03, 6.23559352e+00, 6.21567426e+00, 4.63613532e-02,
       6.11062364e+00, 3.03696050e-03, 6.20504909e+00, 6.23250862e+00,
       6.23250864e+00, 6.20504913e+00, 3.03697767e-03, 6.11062364e+00,
       2.06580747e-01, 6.24680425e+00, 1.61344666e-01, 6.28011762e+00,
       6.28011762e+00, 1.61344651e-01, 6.24680423e+00, 2.06580749e-01,
       2.06580748e-01, 6.24680423e+00, 1.61344665e-01, 6.28011761e+00,
       6.28011761e+00, 1.61344653e-01, 6.24680425e+00, 2.06580749e-01,
       6.11062364e+00, 3.03696402e-03, 6.20504912e+00, 6.23250863e+00,
       6.23250864e+00, 6.20504912e+00, 3.03695820e-03, 6.11062362e+00,
       4.63613687e-02, 6.21567427e+00, 6.23559348e+00, 1.64551808e-03,
       1.64552066e-03, 6.23559351e+00, 6.21567427e+00, 4.63613518e-02,
      